### Ingestion del archivo "country.json"

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de spark

In [0]:
# definimos el schema

countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
countries_df = spark.read.schema(countries_schema).json(f"{bronze_folder_path}/{v_file_date}/country.json")

In [0]:
display(countries_df)

#### Paso 2 - Eliminar columnas no deseadas del DF

In [0]:
# forma 3
from pyspark.sql.functions import col

countries_df = countries_df.drop(col("countryIsoCode"))

In [0]:
# forma 1
countries_dropped_df = countries_df.drop("countryIsoCode")

In [0]:
# forma 2
#countries_dropped_df = countries_df.drop(countries_df["countryIsoCode"])

#### Paso 3 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
countries_final_df = add_ingestion_date(countries_dropped_df) \
                    .withColumnRenamed("countryId", "country_id") \
                    .withColumnRenamed("countryName", "country_name") \
                    .withColumn("environment", lit(v_environment)) \
                    .withColumn("file_date", lit(v_file_date))

In [0]:
display(countries_final_df)

#### Paso 4 - Escribir la salida en formato "Parquet"

In [0]:
countries_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.countries")

In [0]:
%sql
select * from movie_silver.countries

In [0]:
%fs
ls /mnt/moviehistory2025/silver/countries

In [0]:
display(spark.read.format("delta").load("/mnt/moviehistory2025/silver/countries"))

In [0]:
dbutils.notebook.exit("success")